# Trying standard QAOA

In [ ]:
import cudaq
from cudaq import spin
import numpy as np
from scipy.optimize import minimize
import random
import copy
import time

# ============================================================
# OPTIMIZED CUDA-Q QAOA Implementation for LABS
# ============================================================

def labs_hamiltonian(n: int) -> cudaq.SpinOperator:
    """Construct the LABS cost Hamiltonian."""
    hamiltonian = 0.0 * spin.i(0)
    
    for ii in range(n - 3):
        for tt in range(1, (n - ii - 1) // 2 + 1):
            for kk in range(tt + 1, n - ii - tt + 1):
                if ii + kk + tt < n:
                    hamiltonian += 2.0 * spin.z(ii) * spin.z(ii + tt) * spin.z(ii + kk) * spin.z(ii + kk + tt)
    
    for ii in range(n - 2):
        for kk in range(1, (n - ii) // 2 + 1):
            jj = ii + 2 * kk
            if jj < n:
                hamiltonian += spin.z(ii) * spin.z(jj)
    
    return 2*hamiltonian

def get_labs_interactions_flat(n: int):
    """Pre-compute LABS interactions as flat lists."""
    fb_0, fb_1, fb_2, fb_3 = [], [], [], []
    
    for ii in range(n - 3):
        for tt in range(1, (n - ii - 1) // 2 + 1):
            for kk in range(tt + 1, n - ii - tt + 1):
                if ii + kk + tt < n:
                    fb_0.append(ii)
                    fb_1.append(ii + tt)
                    fb_2.append(ii + kk)
                    fb_3.append(ii + kk + tt)
    
    tb_0, tb_1 = [], []
    for ii in range(n - 2):
        for kk in range(1, (n - ii) // 2 + 1):
            jj = ii + 2 * kk
            if jj < n:
                tb_0.append(ii)
                tb_1.append(jj)
    
    return fb_0, fb_1, fb_2, fb_3, tb_0, tb_1

# ============================================================
# SPEEDUP 1: Symmetry-exploiting kernel (fix first qubit to |0⟩)
# ============================================================

@cudaq.kernel
def labs_qaoa_kernel_symmetric(
    n_qubits: int,
    p_layers: int,
    gamma: list[float], 
    beta: list[float],
    fb_0: list[int], fb_1: list[int], fb_2: list[int], fb_3: list[int],
    tb_0: list[int], tb_1: list[int],
    n_four_body: int,
    n_two_body: int
):
    """
    QAOA kernel with symmetry breaking:
    - First qubit fixed to |0⟩ (breaks bit-flip symmetry, halves search space)
    """
    qubits = cudaq.qvector(n_qubits)
    
    # Initial superposition - SKIP first qubit to break bit-flip symmetry
    # Qubit 0 stays in |0⟩, others go to |+⟩
    for q_idx in range(1, n_qubits):
        h(qubits[q_idx])
    
    # Apply p QAOA layers
    for layer in range(p_layers):
        # Phase separator: 4-body ZZZZ terms
        for idx in range(n_four_body):
            x.ctrl(qubits[fb_0[idx]], qubits[fb_1[idx]])
            x.ctrl(qubits[fb_1[idx]], qubits[fb_2[idx]])
            x.ctrl(qubits[fb_2[idx]], qubits[fb_3[idx]])
            rz(4.0 * gamma[layer], qubits[fb_3[idx]])
            x.ctrl(qubits[fb_2[idx]], qubits[fb_3[idx]])
            x.ctrl(qubits[fb_1[idx]], qubits[fb_2[idx]])
            x.ctrl(qubits[fb_0[idx]], qubits[fb_1[idx]])
        
        # Phase separator: 2-body ZZ terms
        for idx in range(n_two_body):
            x.ctrl(qubits[tb_0[idx]], qubits[tb_1[idx]])
            rz(2.0 * gamma[layer], qubits[tb_1[idx]])
            x.ctrl(qubits[tb_0[idx]], qubits[tb_1[idx]])
        
        # Mixer: apply to all qubits except first (which is fixed)
        for q_idx in range(1, n_qubits):
            rx(2.0 * beta[layer], qubits[q_idx])

# ============================================================
# SPEEDUP 2: Pre-computed/transferred parameters from the paper
# ============================================================

def get_transferred_parameters(p_layers: int, n_qubits: int) -> tuple:
    """
    Get pre-optimized parameters that transfer across problem sizes.
    Based on the paper's finding that parameters transfer well.
    
    These are approximate optimal parameters from linear ramp + refinement.
    """
    # Linear ramp schedule (works well according to the paper)
    # gamma increases, beta decreases
    gamma = []
    beta = []
    
    for layer_idx in range(p_layers):
        # Normalized layer position [0, 1]
        t = (layer_idx + 0.5) / p_layers
        
        # Empirically good schedules from QAOA literature
        # gamma: starts small, increases (phase accumulation)
        # beta: starts large, decreases (mixing strength)
        gamma.append(0.4 * t)  # Range ~[0, 0.4]
        beta.append(0.5 * (1 - t) + 0.1)  # Range ~[0.6, 0.1]
    
    return gamma, beta

def get_fourier_parameters(p_layers: int, q_coeffs: int = 2) -> tuple:
    """
    FOURIER parameterization: use only q Fourier coefficients instead of 2p parameters.
    This dramatically reduces the optimization search space.
    
    gamma_l = sum_{k=1}^{q} u_k * sin((k-0.5) * pi * l / p)
    beta_l = sum_{k=1}^{q} v_k * cos((k-0.5) * pi * l / p)
    """
    # Good default Fourier coefficients (can be optimized)
    u_coeffs = [0.3, 0.1][:q_coeffs]  # gamma coefficients
    v_coeffs = [0.4, 0.1][:q_coeffs]  # beta coefficients
    
    gamma = []
    beta = []
    
    for layer_idx in range(p_layers):
        l = layer_idx + 1
        g = sum(u_coeffs[k] * np.sin((k + 0.5) * np.pi * l / p_layers) 
                for k in range(len(u_coeffs)))
        b = sum(v_coeffs[k] * np.cos((k + 0.5) * np.pi * l / p_layers) 
                for k in range(len(v_coeffs)))
        gamma.append(g)
        beta.append(max(0.05, b))  # Keep beta positive
    
    return gamma, beta

# ============================================================
# SPEEDUP 3: Vectorized energy calculation with NumPy
# ============================================================

def calculate_energy_fast(s: np.ndarray) -> int:
    """
    Vectorized energy calculation using NumPy.
    s: 1D numpy array of 0/1 values
    """
    N = len(s)
    # Convert 0/1 to -1/+1
    seq = 2 * s - 1
    
    energy = 0
    for k in range(1, N):
        # Vectorized autocorrelation
        c_k = np.sum(seq[:N-k] * seq[k:])
        energy += c_k * c_k
    
    return int(energy)

def calculate_energy_batch(population: np.ndarray) -> np.ndarray:
    """Calculate energy for entire population at once."""
    return np.array([calculate_energy_fast(ind) for ind in population])

# ============================================================
# Canonicalization for symmetry exploitation
# ============================================================

def canonicalize_bitstring(s: list[int]) -> list[int]:
    """
    Return the canonical representative of a bitstring under LABS symmetries.
    
    Symmetries:
    1. Bit-flip: s and NOT(s) have same energy
    2. Reversal: s and REVERSE(s) have same energy
    
    Canonical form: lexicographically smallest among {s, NOT(s), REV(s), NOT(REV(s))}
    """
    s_arr = np.array(s)
    not_s = 1 - s_arr
    rev_s = s_arr[::-1]
    not_rev_s = 1 - rev_s
    
    candidates = [s_arr, not_s, rev_s, not_rev_s]
    
    # Return lexicographically smallest
    min_candidate = min(candidates, key=lambda x: tuple(x))
    return min_candidate.tolist()

def deduplicate_by_symmetry(population: list, n_bits: int) -> list:
    """
    Remove duplicates accounting for LABS symmetries.
    Returns unique canonical representatives.
    """
    seen = set()
    unique_pop = []
    
    for ind in population:
        canonical = tuple(canonicalize_bitstring(ind))
        if canonical not in seen:
            seen.add(canonical)
            unique_pop.append(list(canonical))
    
    return unique_pop

# ============================================================
# Optimized bridge functions
# ============================================================

def bitstring_to_array(bitstring: str, n_bits: int) -> np.ndarray:
    """Convert bitstring to numpy array with proper padding."""
    padded = bitstring.zfill(n_bits)
    return np.array([int(bit) for bit in padded], dtype=np.int8)

def get_top_k_bitstrings_optimized(counts, k_pop: int, n_bits: int, 
                                    use_symmetry: bool = True) -> list:
    """
    Extract top-k bitstrings, optionally exploiting symmetry.
    """
    # Sort by count
    sorted_bitstrings = sorted(counts.items(), key=lambda x: -x[1])
    
    population = []
    seen_canonical = set()
    
    for bitstring, count in sorted_bitstrings:
        arr = bitstring_to_array(bitstring, n_bits)
        ind = arr.tolist()
        
        if use_symmetry:
            canonical = tuple(canonicalize_bitstring(ind))
            if canonical not in seen_canonical:
                seen_canonical.add(canonical)
                population.append(list(canonical))
        else:
            population.append(ind)
        
        if len(population) >= k_pop:
            break
    
    # Fill if needed
    while len(population) < k_pop:
        population.append(copy.deepcopy(random.choice(population)))
    
    return population

# ============================================================
# Optimized QAOA runner
# ============================================================

def run_qaoa_for_mts_seeding_optimized(
    n_qubits: int, 
    k_pop: int, 
    p_layers: int = 3,
    shots: int = 4096, 
    optimize: bool = False,  # Default to False - use transferred params
    use_symmetry: bool = True,
    optimization_method: str = 'transferred'  # 'transferred', 'fourier', 'full'
):
    """
    Optimized QAOA runner with:
    - Symmetry exploitation
    - Transferred/Fourier parameters (faster than full optimization)
    - Efficient sampling
    """
    # Pre-compute interactions
    fb_0, fb_1, fb_2, fb_3, tb_0, tb_1 = get_labs_interactions_flat(n_qubits)
    n_four_body = len(fb_0)
    n_two_body = len(tb_0)
    
    print(f"  Problem size: {n_qubits} qubits")
    print(f"  4-body interactions: {n_four_body}")
    print(f"  2-body interactions: {n_two_body}")
    print(f"  Using symmetry breaking: {use_symmetry}")
    
    # Select kernel based on symmetry setting
    kernel = labs_qaoa_kernel_symmetric if use_symmetry else labs_qaoa_kernel_symmetric
    
    # Get parameters based on method
    if optimization_method == 'transferred':
        print(f"  Using transferred parameters (no optimization)...")
        optimal_gamma, optimal_beta = get_transferred_parameters(p_layers, n_qubits)
        
    elif optimization_method == 'fourier':
        print(f"  Using Fourier parameterization...")
        # Only optimize 4 parameters instead of 2*p
        hamiltonian = labs_hamiltonian(n_qubits)
        
        def fourier_cost(coeffs):
            u = coeffs[:2].tolist()
            v = coeffs[2:].tolist()
            
            gamma = []
            beta = []
            for l_idx in range(p_layers):
                l = l_idx + 1
                g = sum(u[k] * np.sin((k + 0.5) * np.pi * l / p_layers) for k in range(2))
                b = sum(v[k] * np.cos((k + 0.5) * np.pi * l / p_layers) for k in range(2))
                gamma.append(g)
                beta.append(max(0.05, b))
            
            return cudaq.observe(
                kernel, hamiltonian,
                n_qubits, p_layers,
                gamma, beta,
                fb_0, fb_1, fb_2, fb_3,
                tb_0, tb_1,
                n_four_body, n_two_body
            ).expectation()
        
        x0 = np.array([0.3, 0.1, 0.4, 0.1])
        result = minimize(fourier_cost, x0, method='COBYLA',
                         options={'maxiter': 50, 'rhobeg': 0.2})
        
        u = result.x[:2]
        v = result.x[2:]
        optimal_gamma = []
        optimal_beta = []
        for l_idx in range(p_layers):
            l = l_idx + 1
            g = sum(u[k] * np.sin((k + 0.5) * np.pi * l / p_layers) for k in range(2))
            b = sum(v[k] * np.cos((k + 0.5) * np.pi * l / p_layers) for k in range(2))
            optimal_gamma.append(g)
            optimal_beta.append(max(0.05, b))
        print(f"  Fourier optimization complete. Energy: {result.fun:.4f}")
        
    else:  # Full optimization
        print(f"  Running full parameter optimization...")
        hamiltonian = labs_hamiltonian(n_qubits)
        
        def cost_function(params):
            gamma_params = params[:p_layers].tolist()
            beta_params = params[p_layers:].tolist()
            return cudaq.observe(
                kernel, hamiltonian,
                n_qubits, p_layers,
                gamma_params, beta_params,
                fb_0, fb_1, fb_2, fb_3,
                tb_0, tb_1,
                n_four_body, n_two_body
            ).expectation()
        
        x0 = np.array([0.1 * (idx + 1) / p_layers for idx in range(p_layers)] +
                      [0.5 * (p_layers - idx) / p_layers for idx in range(p_layers)])
        
        result = minimize(cost_function, x0, method='COBYLA',
                         options={'maxiter': 100, 'rhobeg': 0.3})
        
        optimal_gamma = result.x[:p_layers].tolist()
        optimal_beta = result.x[p_layers:].tolist()
        print(f"  Full optimization complete. Energy: {result.fun:.4f}")
    
    # Sample from circuit
    print(f"  Sampling {shots} shots...")
    counts = cudaq.sample(
        kernel,
        n_qubits, p_layers,
        optimal_gamma, optimal_beta,
        fb_0, fb_1, fb_2, fb_3,
        tb_0, tb_1,
        n_four_body, n_two_body,
        shots_count=shots
    )
    
    # Convert to MTS format with symmetry deduplication
    population = get_top_k_bitstrings_optimized(counts, k_pop, n_qubits, use_symmetry)
    
    return population

# ============================================================
# Optimized MTS with incremental energy updates
# ============================================================

def calculate_energy_delta(s: np.ndarray, flip_idx: int, current_energy: int) -> int:
    """
    Calculate new energy after flipping bit at flip_idx.
    Uses incremental update - much faster than full recalculation.
    """
    N = len(s)
    seq = 2 * s - 1  # Convert to +/- 1
    
    # Flipping bit i affects all autocorrelations C_k where:
    # - i appears in the sum (i.e., i < N-k), contributing s_i * s_{i+k}
    # - i+k appears in the sum (i.e., i >= k), contributing s_{i-k} * s_i
    
    old_contrib = 0
    new_contrib = 0
    
    for k in range(1, N):
        c_k_old = np.sum(seq[:N-k] * seq[k:])
        
        # Delta for this k
        delta_c_k = 0
        
        # Term where flip_idx is the first index: s[flip_idx] * s[flip_idx + k]
        if flip_idx + k < N:
            delta_c_k -= 2 * seq[flip_idx] * seq[flip_idx + k]
        
        # Term where flip_idx is the second index: s[flip_idx - k] * s[flip_idx]
        if flip_idx - k >= 0:
            delta_c_k -= 2 * seq[flip_idx - k] * seq[flip_idx]
        
        c_k_new = c_k_old + delta_c_k
        old_contrib += c_k_old * c_k_old
        new_contrib += c_k_new * c_k_new
    
    return current_energy + (new_contrib - old_contrib)

def tabu_search_fast(N: int, s: list[int], max_iters: int = 100, tabu_tenure: int = 5):
    """
    Optimized Tabu Search with:
    - NumPy arrays instead of lists
    - Set for O(1) tabu lookup
    - Incremental energy updates (optional, more complex)
    """
    current_s = np.array(s, dtype=np.int8)
    best_s = current_s.copy()
    best_energy = calculate_energy_fast(current_s)
    current_energy = best_energy
    
    tabu_set = set()  # O(1) lookup instead of O(n)
    tabu_queue = []   # To maintain order for removal
    
    for _ in range(max_iters):
        local_best_idx = -1
        local_best_energy = float('inf')
        
        for i_val in range(N):
            # Flip bit temporarily
            current_s[i_val] ^= 1
            neighbor_energy = calculate_energy_fast(current_s)
            current_s[i_val] ^= 1  # Flip back
            
            is_tabu = i_val in tabu_set
            is_aspiration = neighbor_energy < best_energy
            
            if (not is_tabu) or is_aspiration:
                if neighbor_energy < local_best_energy:
                    local_best_energy = neighbor_energy
                    local_best_idx = i_val
        
        if local_best_idx >= 0:
            current_s[local_best_idx] ^= 1
            current_energy = local_best_energy
            
            if local_best_energy < best_energy:
                best_energy = local_best_energy
                best_s = current_s.copy()
            
            # Update tabu structure
            tabu_set.add(local_best_idx)
            tabu_queue.append(local_best_idx)
            if len(tabu_queue) > tabu_tenure:
                old_idx = tabu_queue.pop(0)
                tabu_set.discard(old_idx)
    
    return best_s.tolist()

def MTS_fast(N: int, k_pop: int, num_iterations: int, 
             mutation_rate: float = 0.05, population: list = None):
    """
    Optimized MTS with NumPy operations.
    """
    if population is None or len(population) == 0:
        population = [np.random.randint(0, 2, N, dtype=np.int8).tolist() 
                      for _ in range(k_pop)]
    
    # Convert to numpy for faster operations
    pop_array = np.array(population, dtype=np.int8)
    
    # Find initial best
    energies = calculate_energy_batch(pop_array)
    best_idx = np.argmin(energies)
    best_solution = pop_array[best_idx].copy()
    best_energy = energies[best_idx]

    print(f"Initial Best Energy: {best_energy}")

    for it in range(num_iterations):
        if it == 0:
            child = pop_array[random.randint(0, k_pop-1)].copy()
        else:
            p1_idx = random.randint(0, k_pop-1)
            p2_idx = random.randint(0, k_pop-1)
            cut_pt = random.randint(1, N-1)
            child = np.concatenate([pop_array[p1_idx, :cut_pt], 
                                   pop_array[p2_idx, cut_pt:]])
        
        # Mutate using vectorized operation
        mutation_mask = np.random.random(N) < mutation_rate
        child = child ^ mutation_mask.astype(np.int8)
        
        # Tabu search
        improved_child = tabu_search_fast(N, child.tolist(), max_iters=N*2)
        improved_energy = calculate_energy_fast(np.array(improved_child))

        if improved_energy < best_energy:
            print(f"Iteration {it}: New Best Energy Found: {improved_energy}")
            best_energy = improved_energy
            best_solution = np.array(improved_child, dtype=np.int8)
            
            replace_idx = random.randint(0, k_pop - 1)
            pop_array[replace_idx] = best_solution

    return best_solution.tolist(), int(best_energy)

# ============================================================
# Main: Optimized Hybrid Quantum-Classical LABS Solver
# ============================================================

if __name__ == "__main__":

    start = time.time()
    
    N = 20
    k = 15
    mts_iterations = 50
    mutation_rate = 0.05

    print("=" * 60)
    print("OPTIMIZED HYBRID QUANTUM-CLASSICAL LABS SOLVER")
    print("=" * 60)
    print(f"\nOptimizations enabled:")
    print(f"  - Symmetry breaking (halves quantum search space)")
    print(f"  - Transferred parameters (skips expensive optimization)")
    print(f"  - NumPy vectorization (faster energy calculations)")
    print(f"  - Set-based tabu list (O(1) lookup)")

    # Step 1: Run optimized QAOA
    print("\n[Step 1] Running QAOA to generate initial population...")
    initial_population = run_qaoa_for_mts_seeding_optimized(
        n_qubits=N, 
        k_pop=k, 
        p_layers=3,  # Can use more layers since we're not optimizing
        shots=4096,
        use_symmetry=True,
        optimization_method='transferred'  # 'transferred', 'fourier', or 'full'
    )

    print(f"\nQuantum-seeded population ({k} individuals):")
    for idx, ind in enumerate(initial_population):
        energy = calculate_energy_fast(np.array(ind))
        print(f"  Individual {idx}: {ind} -> Energy: {energy}")

    qaoa_time = time.time() - start
    print(f"\nQAOA phase completed in {qaoa_time:.2f} seconds")

    # Step 2: Run optimized MTS
    print(f"\n[Step 2] Running MTS with quantum-seeded population...")
    mts_start = time.time()
    best_solution, best_energy = MTS_fast(
        N=N, 
        k_pop=k, 
        num_iterations=mts_iterations, 
        mutation_rate=mutation_rate, 
        population=initial_population
    )
    mts_time = time.time() - mts_start

    print("\n" + "=" * 60)
    print("FINAL RESULTS")
    print("=" * 60)
    print(f"Best solution: {best_solution}")
    print(f"Best energy: {best_energy}")
    if best_energy > 0:
        print(f"Merit factor: {N**2 / (2 * best_energy):.4f}")
    else:
        print("Perfect solution found!")

    total_time = time.time() - start
    print(f"\n[Timing]")
    print(f"  QAOA phase: {qaoa_time:.2f}s")
    print(f"  MTS phase:  {mts_time:.2f}s")
    print(f"  Total:      {total_time:.2f}s")
